<a href="https://colab.research.google.com/github/abduloo7454/XAI-Hyper-GCN/blob/main/Hyper_CSP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hyper-CSP

In [ ]:
from scipy.io import loadmat

data = loadmat('/content/drive/MyDrive/GRAPH EEG/hyperCSP.mat')
data = data['FeaturesAndLabels']

## Experiment 1,2,3,4,5
df = data[:,:30]
label = data[:,-1]

## Experiment 1,2
df = df[:464,:]
label = label[:464]

In [ ]:
df.shape, label.shape

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, KFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Ensure df is a DataFrame and label is a Series
df = pd.DataFrame(df)
label = pd.Series(label)

# 🚀 Feature Scaling (Standardization)
# scaler = StandardScaler()
# df_scaled = scaler.fit_transform(df)  # Scale features

# 🚀 Split dataset into Train (75%), Validation (15%), and Test (10%)
X_train, X_temp, y_train, y_temp = train_test_split(df, label, test_size=0.25, stratify=label, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.4, stratify=y_temp, random_state=42)

# K-Fold Cross Validation (k=5) on the training set
kf = KFold(n_splits=10, shuffle=True, random_state=42)

# 🚀 **Hyperparameter Tuning**
svm_params = {'C': [0.1, 5, 10], 'kernel': ['linear', 'rbf', 'poly']}
knn_params = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 10, 12]}
lda_params = {'solver': ['lsqr', 'eigen'], 'shrinkage': ['auto', None]}  # ✅ FIX: Use correct solvers

# Initialize models
models = {
    "SVM": GridSearchCV(SVC(), svm_params, cv=5, scoring='accuracy'),
    "LDA": GridSearchCV(LinearDiscriminantAnalysis(), lda_params, cv=5, scoring='accuracy'),
    "KNN": GridSearchCV(KNeighborsClassifier(), knn_params, cv=5, scoring='accuracy')
}

# Store results
results = {}

for model_name, model in models.items():
    fold_accuracies = []

    for train_idx, val_idx in kf.split(X_train):
        # Use .iloc to access rows by numerical index:
        X_k_train, X_k_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
        y_k_train, y_k_val = y_train.iloc[train_idx], y_train.iloc[val_idx]

        # Fit the model
        model.fit(X_k_train, y_k_train)
        best_model = model.best_estimator_
        y_pred = best_model.predict(X_k_val)

        acc = accuracy_score(y_k_val, y_pred)
        fold_accuracies.append(acc)

    # Train final model on the entire training set
    best_model.fit(X_train, y_train)

    # Evaluate on validation set
    y_val_pred = best_model.predict(X_val)
    val_accuracy = accuracy_score(y_val, y_val_pred)
    val_precision = precision_score(y_val, y_val_pred, average='macro')
    val_recall = recall_score(y_val, y_val_pred, average='macro')
    val_f1 = f1_score(y_val, y_val_pred, average='macro')

    # Evaluate on test set
    y_test_pred = best_model.predict(X_test)
    test_accuracy = accuracy_score(y_test, y_test_pred)
    test_precision = precision_score(y_test, y_test_pred, average='macro')
    test_recall = recall_score(y_test, y_test_pred, average='macro')
    test_f1 = f1_score(y_test, y_test_pred, average='macro')

    results[model_name] = {
        "Best Params": model.best_params_,
        "Cross-Validation Accuracy": np.mean(fold_accuracies),
        "Validation Accuracy": val_accuracy,
        "Validation Precision": val_precision,
        "Validation Recall": val_recall,
        "Validation F1": val_f1,
        "Test Accuracy": test_accuracy,
        "Test Precision": test_precision,
        "Test Recall": test_recall,
        "Test F1": test_f1
    }

# ✅ Print results properly (Fixing the TypeError)
for model_name, metrics in results.items():
    print(f"\n🔹 {model_name} Performance:")
    for metric, value in metrics.items():
        if isinstance(value, dict):
            print(f"  {metric}: {value}")  # Print dictionary properly
        else:
            print(f"  {metric}: {value:.4f}")  # Format numbers properly